# Data Gathering and Pre-processing

Import the required libraries...

In [3]:
import csv
import pickle
import os
import pandas as pd
import numpy as np
import nfl_data_py as nfl

## Gathering

NFL play by play data found [here](https://github.com/nflverse/nflverse-data/releases/tag/pbp). In this cell we will convert the csv files to pkl files, so they're easier to work with. This part may take a while, as the csv files are pretty big.

In [5]:
# # loop through the play by play csv files
# for i in range(1999, 2023):
#     # open the csv file for the year (i is the year)
#     with open(f'./pbp_csv/play_by_play_{i}.csv', 'r') as f:
#         reader = csv.reader(f)
#         # make pbp_pkl folder before running, if it doesn't exist
#         try:
#             os.makedirs(f'./pbp_pkl')
#         except:
#             pass
#         # create the pickle file for the year, and dump the csv data into it. Store it in pbp_pkl folder
#         pickle.dump(list(reader), open(f'./pbp_pkl/play_by_play_{i}.pkl', 'wb'))

# Load the data
df = pd.read_csv('./pbp_csv/play_by_play_2021.csv')

# Display the first few rows of the dataframe
print(df.head())

# Summary statistics for each column
print(df.describe(include='all'))

# Check for missing values
print('-------------------------------------')
print(df.isnull().sum())

# Drop the 'play_id' column
df.drop('play_id', axis=1, inplace=True)


C:\Users\bnugn\AppData\Local\Temp\ipykernel_1168\2468737981.py:15: DtypeWarning: Columns (179,180,182,183,189,190,193,194,197,198,203,204,205,206,218,219,220,233,234,235,236,237,238,248,249,253,254,255,260,262,263,283,284,301) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./pbp_csv/play_by_play_2021.csv')


   play_id          game_id  old_game_id home_team away_team season_type  \
0        1  2021_01_ARI_TEN   2021091207       TEN       ARI         REG   
1       40  2021_01_ARI_TEN   2021091207       TEN       ARI         REG   
2       55  2021_01_ARI_TEN   2021091207       TEN       ARI         REG   
3       76  2021_01_ARI_TEN   2021091207       TEN       ARI         REG   
4      100  2021_01_ARI_TEN   2021091207       TEN       ARI         REG   

   week posteam posteam_type defteam  ... out_of_bounds  home_opening_kickoff  \
0     1     NaN          NaN     NaN  ...             0                     1   
1     1     TEN         home     ARI  ...             0                     1   
2     1     TEN         home     ARI  ...             0                     1   
3     1     TEN         home     ARI  ...             0                     1   
4     1     TEN         home     ARI  ...             0                     1   

     qb_epa  xyac_epa  xyac_mean_yardage  xyac_median_ya

Next, gather data on team ratings by season from 